In [2]:
import pandas as pd
import numpy as np
import datetime
import time

In [110]:
import datetime

# Example usage
start_time = datetime.datetime(2022, 3, 6, 12, 0, 0)
duration = datetime.timedelta(minutes=60)
sport = 'Biking'
distance = 10000  # in meters
hr = 180

# Convert duration to seconds
duration_seconds = duration.total_seconds()

# Format duration as HH:MM:SS string
duration_str = datetime.datetime.utcfromtimestamp(duration_seconds).strftime('%H:%M:%S')

# Create trackpoints
trackpoints = []
for i in range(0, distance + 100, 100):
    time_delta = datetime.timedelta(seconds=(duration_seconds * i) // distance)
    time = start_time + time_delta
    trackpoint = f"""
            <Trackpoint>
                <Time>{time.isoformat()}Z</Time>
                <Position>
                    <LatitudeDegrees>45.083381</LatitudeDegrees>
                    <LongitudeDegrees>0.0</LongitudeDegrees>
                </Position>
                <HeartRateBpm>
                    <Value>{hr}</Value>
                </HeartRateBpm>
                <DistanceMeters>{i}</DistanceMeters>
            </Trackpoint>
    """
    trackpoints.append(trackpoint)

# Combine trackpoints into a string
trackpoints_str = ''.join(trackpoints)

# Create TCX file string
tcx = f"""<?xml version='1.0' encoding='utf-8'?>
<TrainingCenterDatabase xmlns:ns2="http://www.garmin.com/xmlschemas/UserProfile/v2" xmlns:ns3="http://www.garmin.com/xmlschemas/ActivityExtension/v2" xmlns:ns4="http://www.garmin.com/xmlschemas/ProfileExtension/v1" xmlns:ns5="http://www.garmin.com/xmlschemas/ActivityGoals/v1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" xsi:schemaLocation="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2 http://www.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd">
            <Activities>
              <Activity Sport="{sport}">
      <Id>{start_time.isoformat()}Z</Id>
      <Lap StartTime="{start_time.isoformat()}Z">
        <TotalTimeSeconds>{duration_seconds:.0f}</TotalTimeSeconds>
        <DistanceMeters>{distance}</DistanceMeters>
        <MaximumSpeed>0.0</MaximumSpeed>
        <Calories>0</Calories>
        <AverageHeartRateBpm>
          <Value>{hr}</Value>
        </AverageHeartRateBpm>
        <MaximumHeartRateBpm>
          <Value>{hr}</Value>
        </MaximumHeartRateBpm>
        <Intensity>Active</Intensity>
        <TriggerMethod>Manual</TriggerMethod>
        <Track>
          {trackpoints_str}
        </Track>
      </Lap>
    </Activity>
            </Activities>
            
          </TrainingCenterDatabase>
"""

# Save TCX file
with open('example.tcx', 'w') as f:
    f.write(tcx)


In [111]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': "103516",
    'client_secret': 'b40c81457b89385de62db246c50006bb38f89ef9',
    'refresh_token': 'd7b5ad48dd6d68f1cde587937e96a2aefb87081a',
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

print(my_dataset[0]["name"])
print(my_dataset[0]["map"]["summary_polyline"])

Requesting Token...

Access Token = b9354e49281fd4757b60b5c01998c8901657b5ca

Afternoon Ride
cjdrG???


In [113]:
import pandas as pd
from pandas.io.json import json_normalize

activities = pd.json_normalize(my_dataset)
activities

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state
0,2,Afternoon Ride,9900.0,3564,3564,0.0,Ride,Ride,None,8672207511,...,False,0,0,False,232.0,114752191,1,a8672207511,cjdrG???,2
